# OpenAI Agents SDK with MCP Server

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/03/notebooks/open-meteo-mcp.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/03/notebooks/open-meteo-mcp.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## NOTE: To run this notebook on Colab...

1. Run the pre-requisites cell below to install libraries and NodeJS
2. Open up the Terminal (bottom left of window in Colab)
3. Run the following command: `TRANSPORT=http npx -y open-meteo-mcp-server` - this will start the MCP Server
4. Run the rest of the cells

## Install pre-requisites (Updated OpenAI Agents SDK and NodeJS)

In [ ]:
!uv pip install -q openai-agents==0.7.0
!curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.40.3/install.sh | bash

## Set the OpenAI API Key environment variable

In [8]:
import sys
import os
from dotenv import load_dotenv

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
  load_dotenv()

## MCP Server Setup

In [9]:
import logging

# Suppress MCP client warnings about non-JSON messages from the server
logging.getLogger("mcp").setLevel(logging.CRITICAL)

from agents import Agent, Runner
from agents.mcp import MCPServerStdio, MCPServerStreamableHttp

### List Available Tools

In [ ]:
try:
  async with MCPServerStreamableHttp(
      params = {"url": "http://localhost:3000/mcp"}
      ) as server:
    tools = await server.list_tools()
    print(f"Available tools: {[tool.name for tool in tools]}")
except:
  print("Is the MCP server running? Check at the top of this notebook for instructions.")

Available tools: ['weather_forecast', 'weather_archive', 'air_quality', 'marine_weather', 'elevation', 'flood_forecast', 'seasonal_forecast', 'climate_projection', 'ensemble_forecast', 'geocoding', 'dwd_icon_forecast', 'gfs_forecast', 'meteofrance_forecast', 'ecmwf_forecast', 'jma_forecast', 'metno_forecast', 'gem_forecast']


### Simple Weather Query

In [ ]:
try:
  async with MCPServerStreamableHttp(
    params = {"url": "http://localhost:3000/mcp"}
    ) as server:
    agent = Agent(
        name="Weather Agent",
        model="gpt-5.2",
        instructions="You are a helpful weather assistant. Use the available tools to answer questions about weather forecasts, historical weather data, and air quality. Always provide clear, concise answers.",
        mcp_servers=[server],
    )
    result = await Runner.run(agent, "What's the weather forecast for Minneapolis–St. Paul this week?")
    print(result.final_output)
except:
  print("Is the MCP server running? Check at the top of this notebook for instructions.")

Minneapolis–St. Paul (Twin Cities) forecast for the next 7 days (America/Chicago):

- **Fri Jan 23:** Partly cloudy. **High -8°F / Low -20°F**. Precip **0**. Wind up to **12 mph**.  
- **Sat Jan 24:** Partly cloudy. **High 0°F / Low -16°F**. Precip **0**. Wind up to **6 mph**.  
- **Sun Jan 25:** Partly cloudy. **High 8°F / Low -7°F**. Precip **0**. Wind up to **9 mph**.  
- **Mon Jan 26:** Partly cloudy. **High 13°F / Low -7°F**. Precip **0**. Wind up to **9 mph**.  
- **Tue Jan 27:** Partly cloudy. **High 12°F / Low 3°F**. Precip **0**. Wind up to **12 mph**.  
- **Wed Jan 28:** Partly cloudy. **High 8°F / Low 0°F**. Precip **0**. Wind up to **8 mph**.  
- **Thu Jan 29:** Partly cloudy. **High 10°F / Low 2°F**. Precip **0**. Wind up to **5 mph**.

Overall: **cold, mostly partly cloudy, and dry all week** (no measurable precipitation in the forecast).


### Air Quality Query

In [ ]:
try:
  async with MCPServerStreamableHttp(
    params = {"url": "http://localhost:3000/mcp"}
    ) as server:
    agent = Agent(
        name="Weather Agent",
        model="gpt-5.2",
        instructions="You are a helpful weather assistant. Use the available tools to answer questions about weather forecasts, historical weather data, and air quality. Always provide clear, concise answers.",
        mcp_servers=[server],
    )

    result = await Runner.run(agent, "What's the current air quality in Los Angeles?")
    print(result.final_output)
except:
  print("Is the MCP server running? Check at the top of this notebook for instructions.")

Los Angeles air quality (latest available hour in the model: **2026-01-23 00:00**, America/Los_Angeles):

- **PM2.5:** 48.4 µg/m³  
- **PM10:** 50.1 µg/m³  
- **Ozone (O₃):** 0 µg/m³  
- **Nitrogen dioxide (NO₂):** 71.5 µg/m³  
- **Carbon monoxide (CO):** 896 µg/m³  
- **Sulphur dioxide (SO₂):** 18.8 µg/m³  

If you tell me your nearest neighborhood (e.g., Downtown, Hollywood, Santa Monica), I can check a more location-specific point.


### Historical Weather Data

In [ ]:
try:
  async with MCPServerStreamableHttp(
    params = {"url": "http://localhost:3000/mcp"}
    ) as server:
    agent = Agent(
        name="Weather Agent",
        model="gpt-5.2",
        instructions="You are a helpful weather assistant. Use the available tools to answer questions about weather forecasts, historical weather data, and air quality. Always provide clear, concise answers.",
        mcp_servers=[server],
    )

    result = await Runner.run(agent, "What was the average temperature in Tokyo during January 2024?")
    print(result.final_output)
except:
  print("Is the MCP server running? Check at the top of this notebook for instructions.")

Using ERA5 reanalysis for **Tokyo (35.6895, 139.6917)**, the **average temperature in January 2024** (computed as the mean of each day’s \((T_\max + T_\min)/2\) over Jan 1–31) was:

**≈ 6.1 °C**.
